In [5]:
import cv2
import mediapipe as mp
import math
import os

# Initialize MediaPipe Pose and drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

counter = 0
stage = None
create = None
opname = "output_with_angles.avi"

def calculate_angle(a, b, c):
    """
    Calculate the angle between three points (a, b, c)
    """
    a = [a[0], a[1]]
    b = [b[0], b[1]]
    c = [c[0], c[1]]

    # Vector calculations
    ab = [a[0] - b[0], a[1] - b[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    dot_product = ab[0] * bc[0] + ab[1] * bc[1]
    mag_ab = math.sqrt(ab[0] ** 2 + ab[1] ** 2)
    mag_bc = math.sqrt(bc[0] ** 2 + bc[1] ** 2)

    # Avoid division by zero
    if mag_ab == 0 or mag_bc == 0:
        return 0

    # Calculate angle and convert to degrees
    angle = math.acos(dot_product / (mag_ab * mag_bc))
    return math.degrees(angle)

# Replace the input source to process a video file
cap = cv2.VideoCapture('pushup.mp4')

with mp_pose.Pose(
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7) as pose:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("End of video or error in reading video.")
            break

        image = cv2.resize(image, (640, 480))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        lmList = []
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            for id, lm in enumerate(results.pose_landmarks.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])

        if len(lmList) != 0:
            # Get coordinates for left arm
            shoulder = lmList[11]  # Left shoulder
            elbow = lmList[13]     # Left elbow
            wrist = lmList[15]     # Left wrist

            # Calculate angle at the elbow
            angle = calculate_angle(shoulder[1:], elbow[1:], wrist[1:])

            # Display the angle on the video
            cv2.putText(image, str(int(angle)), (elbow[1], elbow[2] - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            # Push-up logic based on angle
            if angle < 90:
                stage = "down"
            if angle > 160 and stage == "down":
                stage = "up"
                counter += 1
                print(f"Push-up count: {counter}")
                os.system(f"echo '{counter}' | festival --tts")

        # Display counter on video
        text = f"Push-ups: {counter}"
        cv2.putText(image, text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Save processed frames to output video
        if create is None:
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            create = cv2.VideoWriter(opname, fourcc, 30, (image.shape[1], image.shape[0]), True)

        create.write(image)
        cv2.imshow('Push-up Detection with Angle', image)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

cap.release()
if create is not None:
    create.release()
cv2.destroyAllWindows()


Push-up count: 1
Push-up count: 2
Push-up count: 3
Push-up count: 4
Push-up count: 5
Push-up count: 6
Push-up count: 7
Push-up count: 8
Push-up count: 9
